In [1]:
# parameters

truncate = None
location = '/root/.cache/keypoints'
clear_cache = False

In [2]:
import joblib

from joblib import Memory
memory = Memory(location,verbose=0)
if clear_cache:
    memory.clear()

In [3]:
from action_transformer import dataset_mod

In [4]:
from pathlib import Path


folder_video_keypoints = "../data/keypoints/as_arrays" 
folder_video_keypoints = str(Path(folder_video_keypoints).resolve())
# with respect to the notebook's directory, all the notebooks are launched with respect to their folder's location
# using papermill or ploomber engine

dataset_mod.VideoKeyPointDataset.memory = memory

video_kpt_dataset = dataset_mod.VideoKeyPointDataset(folder_video_keypoints,with_frame_padding=False,truncate=truncate)


In [5]:
if truncate is not None:
    assert len(video_kpt_dataset) == min(truncate,len(video_kpt_dataset.folder_video_keypoints))
print(len(video_kpt_dataset))

599


In [33]:
from ploomber_engine import execute_notebook

In [42]:
from ploomber.tasks import NotebookRunner
from ploomber import DAG
from ploomber.products import File
dag = DAG()
ff = NotebookRunner(Path(path_cache_setup),File(path_cache_setup.replace(".py",".ipynb")),dag=dag)
#dag.build()

In [68]:
# fill the cache and print informations about caching in other notebook
from ploomber_engine.ipython import PloomberClient
path_cache_setup = "./performance_tests/caching_tests_and_metrics.py"
path_cache_setup = str(Path(path_cache_setup).resolve())

In [71]:


import jupytext
path_cache_setup_as_notebook = path_cache_setup.replace(".py",".ipynb")
jupytext.write(nb=jupytext.read(path_cache_setup),
               fp =path_cache_setup_as_notebook)


client = PloomberClient.from_path(path_cache_setup_as_notebook,remove_tagged_cells="notebook_call")
jupytext.write(nb_node,path_cache_setup.replace(".py",".ipynb"))
namespace = client.get_namespace(dict(video_kpt_dataset=video_kpt_dataset,clear_cache=clear_cache,location=location))
import os
os.remove(path_cache_setup_as_notebook)

Executing cell: 9: 100%|████████████████████████| 10/10 [00:00<00:00, 10.24it/s]


In [31]:
import numpy as np
all_labels = list(np.unique([el["class_name"] for el in video_kpt_dataset]))

label_to_idx = {label:idx for (idx,label) in enumerate(all_labels)}

transform_labels = lambda label : label_to_idx[label]
video_kpt_dataset.with_frame_padding = True
video_kpt_dataset.transform_labels = transform_labels


In [46]:
res = video_kpt_dataset[3]["class_name"]
type(res)

int

In [47]:
import torch

def collatefn(batch):
    batch = {k: [dic[k] for dic in batch] for k in batch[0]}
    nb_frames,kpts,is_detection_present,classes_idx = batch.values()
    max_nb_frames = max(nb_frames) 
    # we compute the maximum over the batch for some extra computation savings
    kpts = torch.tensor(kpts)
    kpts = kpts.reshape(kpts.shape[0],kpts.shape[1],-1)
    is_detection_present = torch.tensor(is_detection_present)
    
    kpts = kpts[:,:max_nb_frames]
    is_detection_present = is_detection_present[:,:max_nb_frames]
    #nb_frames = np.array(nb_frames)
    class_names = torch.tensor(classes_idx,dtype=torch.long)
    return kpts,is_detection_present,class_names

from torch.utils.data import DataLoader

video_dataloader = DataLoader(video_kpt_dataset,shuffle=True,collate_fn=collatefn,batch_size=10)

In [48]:
if False:
    res = next(iter(video_dataloader))
    res[0].shape,res[1].shape

In [49]:
type(video_dataloader.sampler) == torch.utils.data.sampler.RandomSampler

True

In [50]:
#sanity check test
#res[0][0].numpy()
#video_kpt_dataset[0][0]#res[0][0].numpy()
#video_kpt_dataset[0][0]#res[0][0].numpy()
#video_kpt_dataset[0][0]
unshuffled = False
if unshuffled:
    loader_iter = iter(d_loader)
    res = next(loader_iter)
    assert np.all(res[0][0][6].numpy() == video_kpt_dataset[0][0][6])

In [51]:
for el in video_dataloader:
    break

/tmp/ipykernel_5218/1364585169.py:10: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  is_detection_present = torch.tensor(is_detection_present)
